# Differentiable Logic Cellular Automata: from Game of Life to Pattern Generation

This notebook contains code to reproduce experiments and figures for the
"Differentiable Logic Cellular Automata: from Game of Life to Pattern
Generation" article.

## Experiment: Checkerboard (Asynchronous Training)

This notebook trains a DiffLogic CA to generate a checkerboard pattern using asynchronous updates (random update masks). It includes training, visualization, and robustness/self-healing tests under asynchronous dynamics.

This notebook is standalone and includes the shared setup code from the original DiffLogic CA notebook.


Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use
this file except in compliance with the License. You may obtain a copy of the
License at

https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or
agreed to in writing, software distributed under the License is distributed on
an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express
or implied. See the License for the specific language governing permissions and
limitations under the License.

**Note for the reader (as of March 20, 2025):**: We tested reproducibility of the results reported in
this article using publicly available Colab notebooks with T4 GPUs. Determinism
is ensured by setting the os.environ['XLA_FLAGS'] = '
--xla_gpu_deterministic_ops=true' flag, which resolves non-determinism caused by
specific JAX optimization routines, though at the cost of speed and efficiency.

In [ ]:
# We tested reproducibility on jax version 0.4.33

!pip install jax[cuda12_pip]==0.4.33
!pip install jaxlib==0.4.33

# You might need to restart the kernel

In [ ]:
# @title Patch extraction backend
# Set to False on older GPUs that fail cuDNN autotuning.
USE_CUDNN = True


In [1]:
"""Reproducibility:

This flag is necessary for ensuring reproducibility on Colab.
Removing it significantly boosts performance (5-10x) but compromises
result consistency due to inherent GPU non-determinism.
"""

import os

# os.environ['XLA_FLAGS'] = ' --xla_gpu_deterministic_ops=true'


In [2]:
# @title Imports and Notebook Utilities
from collections import namedtuple
from datetime import datetime
from functools import partial
import io
import itertools
from base64 import b64decode
from typing import List, Optional, Sequence, Tuple

from einops import rearrange
import flax.linen as nn
import flax.linen as nn
from IPython.display import HTML, Image, clear_output
import jax
print(jax.__version__)
from jax import grad
from jax.lax import conv_general_dilated_patches
import jax.numpy as jnp
import jax.random as random
from jax.tree_util import tree_leaves, tree_map
from matplotlib import colors
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import matplotlib.style as style
from matplotlib.ticker import MaxNLocator
import numpy as np
import optax
import PIL
import PIL.Image, PIL.ImageDraw, PIL.ImageFont, PIL.ImageOps
import requests


os.environ['FFMPEG_BINARY'] = 'ffmpeg'

0.8.1


In [3]:
# @title Hyperparameters

# Index of the pass-through gate used for network initialization.
PASS_THROUGH_GATE = 3
DEFAULT_PASS_VALUE = 10.0

# Number of possible gates with 2 inputs and 1 output.
NUMBER_OF_GATES = 16

"""
Fire rate for asynchronicity.

60% of cells will perform the update,
40% will have the update masked.
"""
FIRE_RATE = 0.6

TARGET_EMOJI = "🦎"
TARGET_SIZE_EMOJI = 20
TARGET_SIZE_G = 16
TARGET_SIZE_ASYNC = 14
TARGET_SIZE = 16

In [4]:
# @title Utils and Plotting


def zoom(img, scale=4):
  img = np.repeat(img, scale, 0)
  img = np.repeat(img, scale, 1)
  return img


def load_byte(byte, max_size=TARGET_SIZE):
  img = PIL.Image.open(io.BytesIO(byte))
  img.thumbnail((max_size, max_size), PIL.Image.LANCZOS)
  img = np.float32(img) / 255.0
  # premultiply RGB by Alpha
  img[..., :3] *= img[..., 3:]
  return img

def visualize(frames, namefile, size, negate=1):
  # Create and save animation
  fig, ax = plt.subplots(figsize=(size, size))
  ax.set_xticks([])
  ax.set_yticks([])

  def animate(frame):
    ax.clear()
    ax.set_xticks([])
    ax.set_yticks([])

    if negate:
      frame = 1 - frame

    return [ax.imshow(frame, cmap='binary')]

  anim = animation.FuncAnimation(fig, animate, frames=frames, interval=200, blit=True)

  # Save the gif
  writer = animation.PillowWriter(fps=2)
  anim.save(namefile, writer=writer)
  plt.close()


def plot_show(img, negate=1):
  if negate:
    img = 1 - img
  fig, ax = plt.subplots(1, 1, figsize=(5, 5))
  ax.imshow(img, cmap='binary')
  ax.axis('off')
  plt.tight_layout()
  plt.show()


# Plot the histogram related to the statistics of the gates.
def plot_hist_gates(
    net, save_path=None, title='Distribution of Gates (Sorted)'
):
  gate_types = [
      'FALSE',
      'AND',
      'A AND (NOT B)',
      'A',
      '(NOT A) AND B',
      'B',
      'XOR',
      'OR',
      'NOR',
      'XNOR',
      'NOT B',
      'A OR (NOT B)',
      'NOT A',
      '(NOT A) OR B',
      'NAND',
      'TRUE',
  ]

  gate_counts = np.bincount(net, minlength=len(gate_types))
  sorted_indices = np.argsort(gate_counts)
  sorted_counts = gate_counts[sorted_indices]
  sorted_gate_types = [gate_types[i] for i in sorted_indices]
  fig, ax = plt.subplots(figsize=(12, 6), dpi=100)

  ax.barh(
      sorted_gate_types,
      sorted_counts,
      color='#2E86C1',
      alpha=0.7,
      edgecolor='white',
      linewidth=1,
  )

  ax.set_xlabel('#Gates', fontsize=12, fontweight='bold')
  ax.set_ylabel('Type of gate', fontsize=12, fontweight='bold')
  ax.set_title(
      'Distribution of Gates (Sorted)', fontsize=14, fontweight='bold', pad=20
  )

  plt.style.use('seaborn-v0_8-darkgrid')
  ax.grid(True, linestyle='--', alpha=0.7, axis='x')
  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)
  ax.xaxis.set_major_locator(MaxNLocator(integer=True))
  ax.set_facecolor('#f8f9fa')
  fig.set_facecolor('white')
  plt.tight_layout()

  if save_path:
    with open(save_path, 'wb') as f:
      plt.savefig(f, format='svg')
  plt.show()


# Plot the training loss functions.
def plot_training_progress(
    loss_train, loss_test, compute_every, save_path=None
):
  plt.style.use('seaborn-v0_8-darkgrid')

  fig, ax = plt.subplots(figsize=(10, 6), dpi=100)

  ax.grid(True, color='gray', linestyle='-', linewidth=0.8, alpha=0.3)

  ax.plot(
      loss_train,
      color='#2E86C1',
      linewidth=2,
      label='Soft Gates Loss',
      alpha=0.9,
  )

  ax.plot(
      np.arange(0, len(loss_train), compute_every),
      loss_test,
      color='#E74C3C',
      linestyle='--',
      linewidth=2,
      label='Hard Gates Loss',
      alpha=0.9,
  )

  ax.grid(True, linestyle='--', alpha=0.7)

  ax.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
  ax.set_ylabel('Loss Value', fontsize=12, fontweight='bold')
  ax.set_title(
      'Training Progress - Soft vs Hard Gates',
      fontsize=14,
      fontweight='bold',
      pad=20,
  )

  ax.legend(
      frameon=True, fancybox=True, shadow=True, fontsize=10, loc='upper right'
  )

  for spine in ax.spines.values():
    spine.set_linewidth(1.5)

  ax.set_facecolor('#f8f9fa')
  fig.set_facecolor('white')

  plt.tight_layout()

  if save_path:
    with open(save_path, 'wb') as f:
      plt.savefig(f, format='svg')

  plt.show()

In [5]:
# @title Model definition


def get_moore_connections(key):
  """Generate Moore neighborhood connections for a 9x1 vector.

  Center element is at index 4 and connects to all other elements.
  """
  neighbors = jnp.array([0, 1, 2, 3, 5, 6, 7, 8])
  a = neighbors
  b = jnp.full_like(neighbors, 4)
  perm = jax.random.permutation(key, neighbors.shape[0])
  a = a[perm]
  b = b[perm]
  return a, b


# From https://github.com/Felix-Petersen/difflogic/tree/main/difflogic
def get_unique_connections(in_dim, out_dim, key):
  assert (
      out_dim * 2 >= in_dim
  )  # Number of neurons must not be smaller than half of inputs
  x = jnp.arange(in_dim)
  # Take pairs (0, 1), (2, 3), (4, 5), ...
  a = x[::2]
  b = x[1::2]
  m = min(a.shape[0], b.shape[0])
  a = a[:m]
  b = b[:m]
  # If needed, add pairs (1, 2), (3, 4), (5, 6), ...
  if a.shape[0] < out_dim:
    a_ = x[1::2]
    b_ = x[2::2]
    m = min(a_.shape[0], b_.shape[0])
    a = jnp.concatenate([a, a_[:m]])
    b = jnp.concatenate([b, b_[:m]])
  # If still needed, add pairs with larger offsets
  offset = 2
  while out_dim > a.shape[0] and offset < in_dim:
    a_ = x[:-offset]
    b_ = x[offset:]
    a = jnp.concatenate([a, a_])
    b = jnp.concatenate([b, b_])
    offset += 1

  if a.shape[0] >= out_dim:
    a = a[:out_dim]
    b = b[:out_dim]
  else:
    raise ValueError(
        f'Could not generate enough unique connections: {a.shape[0]} <'
        f' {out_dim}'
    )

  # Random permutation
  perm = jax.random.permutation(key, out_dim)
  a = a[perm]
  b = b[perm]

  return a, b


def bin_op_all_combinations(a, b):
  # Implementation of binary operations between two inputs for all the different operations
  return jnp.stack(
      [
          jnp.zeros_like(a),
          a * b,
          a - a * b,
          a,
          b - a * b,
          b,
          a + b - 2 * a * b,
          a + b - a * b,
          1 - (a + b - a * b),
          1 - (a + b - 2 * a * b),
          1 - b,
          1 - b + a * b,
          1 - a,
          1 - a + a * b,
          1 - a * b,
          jnp.ones_like(a),
      ],
      axis=-1,
  )



def bin_op_s(a, b, i_s):
  # Compute all possible operations
  combinations = bin_op_all_combinations(
      a, b
  )  # Shape: (n_gate, n_possible_gates, 16)
  """
    Calculate the weighted sum of all possible gate operations.
    During training (soft decoding), the weights are the probabilities
    of each gate type. During inference (hard decoding), the weights
    are a one-hot vector representing the selected gate type.
    """
  result = jax.numpy.sum(combinations * i_s[None, ...], axis=-1)
  return result



def decode_soft(weights):
  # From the weights vector compute the probability distribution of choosing each gate using softmax
  return nn.softmax(weights, axis=-1)


def decode_hard(weights):
  return jax.nn.one_hot(
      jnp.argmax(weights, axis=-1), 16
  )  # Return the gate with maximum probability.


# Initialize gates as pass through gate
def init_gates(
    n,
    num_gates=NUMBER_OF_GATES,
    pass_through_gate=PASS_THROUGH_GATE,
    default_pass_value=DEFAULT_PASS_VALUE,
):
  """Initializes a gate matrix with default pass-through values.

  Args:
      n: The number of rows in the gate matrix.
      num_gates: The number of gates (columns). Defaults to NUMBER_OF_GATES.
      pass_through_gate: The index of the pass-through gate column. Defaults to
        PASS_THROUGH_GATE.
      default_pass_value: The default value for the pass-through gate. Defaults
        to DEFAULT_PASS_VALUE.

  Returns:
      An array representing the initialized gate matrix.
  """
  gates = jnp.zeros((n, num_gates))
  gates = gates.at[:, pass_through_gate].set(default_pass_value)
  return gates


def init_gate_layer(key, in_dim, out_dim, connections):

  # With 'random' connections the input of each gate are sampled randomly from the previous layer.
  if connections == 'random':
    key1, key2 = jax.random.split(key)

    c = jax.random.permutation(key2, 2 * out_dim) % in_dim
    c = jax.random.permutation(key1, in_dim)[c]

    c = c.reshape(2, out_dim)

    indices_a = c[0, :]
    indices_b = c[1, :]

  # With 'unique' connections each gate will have a different pair of inputs.
  elif connections == 'unique':
    indices_a, indices_b = get_unique_connections(in_dim, out_dim, key)

  # With 'first_kernel' the connections are specifically designed to mimic the Moore neighborhood.
  elif connections == 'first_kernel':
    indices_a, indices_b = get_moore_connections(key)
  else:
    raise ValueError(f'Connection type {connections} not implemented')

  wires = [indices_a, indices_b]
  gate_logits = init_gates(out_dim)
  return gate_logits, wires


def init_logic_gate_network(hyperparams, params, wires, key):
  for i, (in_dim, out_dim) in enumerate(
      zip(hyperparams['layers'][:-1], hyperparams['layers'][1:])
  ):
    key, subkey = jax.random.split(key)
    gate_logits, gate_wires = init_gate_layer(
        subkey, int(in_dim), int(out_dim), hyperparams['connections'][i]
    )
    params.append(gate_logits)
    wires.append(gate_wires)


def init_perceive_network(hyperparams, params, wires, key):
  for i, (in_dim, out_dim) in enumerate(
      zip(hyperparams['layers'][:-1], hyperparams['layers'][1:])
  ):
    key, subkey = jax.random.split(key)
    gate_logits, gate_wires = init_gate_layer(
        subkey, int(in_dim), int(out_dim), hyperparams['connections'][i]
    )
    """
    Replicate the gate logits for each of the 'n_kernels' perception kernels.
    This allows for parallel computation of the perception module,
    as all kernels share the same underlying structure and wiring.
    """
    params.append(
        gate_logits.repeat(hyperparams['n_kernels'], axis=0).reshape(
            hyperparams['n_kernels'], out_dim, NUMBER_OF_GATES
        )
    )
    wires.append(gate_wires)


# In the current implementation all the kernels share the same connection scheme
def init_diff_logic_ca(hyperparams, key):

  key, subkey = jax.random.split(key)
  """Initialize parameters for the update and perceive networks.

   Each network's parameters will be stored as a list of gate logits.

   'update':  List of gate logits for the update network.
              Each element is a JAX array with shape (out_dim, 16),
              where out_dim is the output dimension of the layer.
   'perceive': List of gate logits for the perceive network.
               Each element is a JAX array with shape (n_kernels * out_dim, 16),
               where n_kernels is the number of perception kernels
               and out_dim is the output dimension of the layer.
  """
  params = {'update': [], 'perceive': []}

  """Initialize wiring for the update and perceive networks.

   Each network's wiring will be stored as a list of connection indices.

   'update':  List of connection indices for the update network.
              Each element is a tuple of two JAX arrays (indices_a, indices_b)
              representing the input indices for each gate.
   'perceive': List of connection indices for the perceive network.
               The perceive kernels share the same wiring, so this list
               contains a single tuple of connection indices (indices_a, indices_b).
  """
  wires = {'update': [], 'perceive': []}

  # Initialize the gate
  init_logic_gate_network(
      hyperparams['update'], params['update'], wires['update'], subkey
  )

  key, subkey = jax.random.split(key)

  # Initialize perceive vector
  init_perceive_network(
      hyperparams['perceive'], params['perceive'], wires['perceive'], subkey
  )

  return params, wires


def run_layer(logits, wires, x, training):
  """Args:

      x: input vector, shape (input_dim, 1).
      wires: wire configuration, shape (out_dim, 1).
      logits: gate parameters, shape (n_out, 16).

  Returns:
      If training is True, the expected gate values.
      If training is False, the gates with maximum probability.
  """

  a = x[..., wires[0]]
  b = x[..., wires[1]]
  logits = jax.lax.cond(training, decode_soft, decode_hard, logits)
  out = bin_op_s(a, b, logits)
  return out


def run_update(params, wires, x, training):
  for g, c in zip(params, wires):
    x = run_layer(g, c, x, training)
  return x


def run_perceive(params, wires, x, training):
  """Applies a perception layer to a patch.

  Args:
      params: List of kernel parameters for each layer.
      wires: List of wire configurations for each layer.
      x: Input patch, shape [batch_size, patch_size, channel_size].
      training: Boolean indicating training mode.

  Returns:
      Output feature vector, shape [batch_size, channel_size].
  """

  # Apply each layer using vmap for kernel parallelism.
  run_layer_map = jax.vmap(run_layer, in_axes=(0, None, 0, None))
  x_prev = x
  x = x.T  # [channel_size, batch_size, patch_size]

  """
    Duplicate 'x' to create n_kernels copies for the first layer, which all share the same input.
    Subsequent layers receive unique inputs.
    """
  x = jnp.repeat(
      x[None, ...], params[0].shape[0], axis=0
  )  # [n_kernels, channel_size, batch_size, patch_size]

  # Iterate through layers, applying kernels and wire configurations.
  for g, c in zip(params, wires):
    x = run_layer_map(g, c, x, training)

  x = rearrange(
      x, 'k c s -> (c s k)'
  )  # [channel_size * patch_size * n_kernels]

  return jnp.concatenate(
      [x_prev[4, :], x], axis=-1
  )  # Concatenate the original input.


def run_circuit(params, wires, x, training):
  x = run_perceive(params['perceive'], wires['perceive'], x, training)
  x = run_update(params['update'], wires['update'], x, training)
  return x

In [6]:
# Test cases
key = jax.random.PRNGKey(0)
key_init, key = jax.random.split(key, 2)
n_kernels = 4
layers = [128] * 10 + [64, 32, 16]
connections = ['unique'] * len(layers)

"""Recap: Structure of the hyperparameters:

'update' is referring to the diff logic network of the update.
- Layers:  number of gates per layer (with exception of the first layer
           which represent the number of inputs)
- Connections: specify the topology of the connections, how the gates
           are connected. It can be mainly 'random', 'unique' or 'first_kernel'
           which will be used to the first kernel layer.

'perceive' is referring to the diff logic networks of the perception
          (or "convolutional kernels").
- Layers:  number of gates per layer (with exception of the first layer which
           represent the number of inputs)
- Connections: specify the topology of the connections, how the gates are connected.
           It can be mainly 'random', 'unique' or 'first_kernel' which is used
           only for the first kernel layer to enforce connections that mimic the cell-interactions.

For simplicity, all perception kernels share the same structure (type of connections,
number of layers, and number of gates per layer). They are evaluated in parallel.

"""

hyperparams = {
    'update': {'layers': layers, 'connections': connections},
    'perceive': {
        'n_kernels': n_kernels,
        'layers': [9, 8, 4, 2, 1],
        'connections': ['first_kernel', 'unique', 'unique', 'unique'],
    },
}

training = True
params, wires = init_diff_logic_ca(hyperparams, key_init)
print(
    params['perceive'][0]
    .reshape((
        hyperparams['perceive']['n_kernels'],
        hyperparams['perceive']['layers'][1],
        16,
    ))
    .shape
)
grid_ch_dim = 16
inp = jnp.ones((3 * 3, grid_ch_dim)).astype(jnp.float32)
x = run_circuit(params, wires, inp, training)
print('INPUT SHAPE', inp.shape)
print('OUTPUT SHAPE', x.shape)

# The output of the circuit is the new state for the central cell with shape (1, grid_ch_dim)
assert x.shape == (1, grid_ch_dim)
print(jax.devices())


W0205 22:53:42.914490  180749 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W0205 22:53:42.916361  180681 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.


(4, 8, 16)
INPUT SHAPE (9, 16)
OUTPUT SHAPE (1, 16)
[CudaDevice(id=0)]


In [7]:
# @title get_grid_patches

import os

# If not already set in the notebook, allow env var override.
if "USE_CUDNN" not in globals():
  _use_cudnn_env = os.environ.get("DLCA_USE_CUDNN", "1").lower()
  USE_CUDNN = _use_cudnn_env not in ("0", "false", "no")

@partial(jax.jit, static_argnums=(1,2))
def _get_grid_patches_cudnn(grid, patch_size, channel_dim, periodic):
  # Given a grid NxN it creates a list of (patch_size x patch_size, channel_dim) patches.

  pad_size = (patch_size - 1) // 2

  # Two possible modes exist: with or without periodic boundary conditions.
  padded_grid = jax.lax.cond(
      periodic,
      lambda g: jnp.pad(
          g, ((pad_size, pad_size), (pad_size, pad_size), (0, 0)), mode="wrap"
      ),
      lambda g: jnp.pad(
          g,
          ((pad_size, pad_size), (pad_size, pad_size), (0, 0)),
          mode="constant",
          constant_values=0,
      ),
      grid,
  )
  padded_grid = jnp.expand_dims(padded_grid, axis=0)
  patches = conv_general_dilated_patches(
      padded_grid,
      filter_shape=(patch_size, patch_size),
      window_strides=(1, 1),
      padding="VALID",
      dimension_numbers=("NHWC", "OIHW", "NHWC"),
  )[0]

  # Rearrange to have (list, patch_size x patch_size, channel_dim)
  patches = rearrange(patches, "x y (c f) -> (x y) f c", c=channel_dim)
  return patches

@partial(jax.jit, static_argnums=(1,2))
def _get_grid_patches_fallback(grid, patch_size, channel_dim, periodic):
  # Given a grid NxN it creates a list of (patch_size x patch_size, channel_dim) patches.

  pad_size = (patch_size - 1) // 2

  # Two possible modes exist: with or without periodic boundary conditions.
  padded_grid = jax.lax.cond(
      periodic,
      lambda g: jnp.pad(
          g, ((pad_size, pad_size), (pad_size, pad_size), (0, 0)), mode="wrap"
      ),
      lambda g: jnp.pad(
          g,
          ((pad_size, pad_size), (pad_size, pad_size), (0, 0)),
          mode="constant",
          constant_values=0,
      ),
      grid,
  )

  # Extract patches without cuDNN to avoid GPU autotune failures on older GPUs.
  n = grid.shape[0]
  ys = jnp.arange(n, dtype=jnp.int32)
  xs = jnp.arange(n, dtype=jnp.int32)

  def slice_at(y, x):
    return jax.lax.dynamic_slice(
        padded_grid, (y, x, 0), (patch_size, patch_size, channel_dim)
    )

  patches = jax.vmap(lambda y: jax.vmap(lambda x: slice_at(y, x))(xs))(ys)

  # Rearrange to have (list, patch_size x patch_size, channel_dim)
  patches = patches.reshape(n * n, patch_size * patch_size, channel_dim)
  return patches

def get_grid_patches(grid, patch_size, channel_dim, periodic, use_cudnn=USE_CUDNN):
  if use_cudnn:
    return _get_grid_patches_cudnn(grid, patch_size, channel_dim, periodic)
  return _get_grid_patches_fallback(grid, patch_size, channel_dim, periodic)

def get_grid_patches_auto(grid, patch_size, channel_dim, periodic):
  global USE_CUDNN
  if USE_CUDNN:
    try:
      return _get_grid_patches_cudnn(grid, patch_size, channel_dim, periodic)
    except Exception:
      USE_CUDNN = False
  return _get_grid_patches_fallback(grid, patch_size, channel_dim, periodic)


In [8]:
# Test

key = random.PRNGKey(42)
N = 3
C = 1
patch_size = 3
grid = random.randint(key, (N, N, C), minval=0, maxval=2).astype(jnp.float32)
patches = get_grid_patches(
    grid, patch_size=patch_size, channel_dim=grid.shape[-1], periodic=1
)
print(f"Grid\n {grid[..., 0]}")
print(patches[-1, :, 1].reshape(1, 3, 3))
print(patches[-1:, 1].reshape(-1))

assert patches.shape == (N * N, patch_size * patch_size, C)

Grid
 [[0. 0. 1.]
 [1. 1. 1.]
 [1. 1. 0.]]
[[[1. 1. 1.]
  [1. 0. 1.]
  [0. 1. 0.]]]
[1.]


In [9]:
# @title run iterations


# patches = [batch_size, n_patches, patch_size x patch_size, channels]
def v_run_circuit_patched(patches, params, wires, training):
  run_circuit_patch = jax.vmap(
      run_circuit, in_axes=(None, None, 0, None)
  )  # vmap over the patches
  x = run_circuit_patch(params, wires, patches, training)
  return x

@jax.jit
def run_async(grid, params, wires, training, periodic, key):
  patches = get_grid_patches(
      grid, patch_size=3, channel_dim=grid.shape[-1], periodic=periodic
  )
  x_new = v_run_circuit_patched(patches, params, wires, training)
  x_new = x_new.reshape(*grid.shape)
  update_mask_f32 = (
      jax.random.uniform(key, x_new[..., :1].shape) <= FIRE_RATE
  ).astype(jax.numpy.float32)
  x = grid * (1 - update_mask_f32) + x_new * update_mask_f32
  return x

@jax.jit
def run_sync(grid, params, wires, training, periodic):
  patches = get_grid_patches(
      grid, patch_size=3, channel_dim=grid.shape[-1], periodic=periodic
  )
  x_new = v_run_circuit_patched(patches, params, wires, training)
  x_new = x_new.reshape(*grid.shape)
  return x_new


@partial(jax.jit, static_argnames=['num_steps', 'periodic', 'async_training'])
def run_iter_nca(grid, params, wires, training, periodic, num_steps, async_training, key):
  def body_fn(carry, i):
    grid, key = carry
    if async_training:
      key, subkey = jax.random.split(key)
      x = run_async(grid, params, wires, training, periodic, subkey)
    else:
      x = run_sync(grid, params, wires, training, periodic)
    return (x, key), 0

  (grid, key), _ = jax.lax.scan(
      body_fn, (grid, key), jnp.arange(0, num_steps, 1)
  )
  return grid


v_run_iter_nca = jax.vmap(
    run_iter_nca, in_axes=(0, None, None, None, None, None, None, None)
)

In [10]:
"""Test

This test verifies that the output dimensions
of the diffLogic CA correspond to the grid dimensions.
"""

batch_size = 1
N = 5
C = 8
n_kernels = 4
layers = [128] * 10 + [64, 32, 16, C]
connections = ['random'] * len(layers)
key = jax.random.PRNGKey(42)

hyperparams = {
    'update': {'layers': layers, 'connections': connections},
    'perceive': {
        'n_kernels': n_kernels,
        'layers': [9, 8, 4, 2, 1],
        'connections': ['first_kernel', 'unique', 'unique', 'unique'],
    },
}
params, wires = init_diff_logic_ca(hyperparams, key)
grid = random.randint(key, (batch_size, N, N, C), minval=0, maxval=2).astype(
    jnp.float32
)

print('Input shape ', grid.shape)

# Run for 10 iterations
num_steps = 10
key, subkey = jax.random.split(key)
x = v_run_iter_nca(grid, params, wires, True, True, num_steps, False, subkey)

print('Output shape', x.shape)

assert x.shape == (batch_size, N, N, C)

Input shape  (1, 5, 5, 8)
Output shape (1, 5, 5, 8)


In [ ]:
TrainState = namedtuple('TrainState', 'param opt_state key')

def init_state(hyperparams, opt, seed):
  key = random.PRNGKey(seed)
  key, subkey = random.split(key, 2)
  params, wires = init_diff_logic_ca(hyperparams, subkey)
  opt_state = opt.init(params)
  return TrainState(params, opt_state, key), wires


def loss_f(params, wires, train_x, train_y, periodic, num_steps, async_training, key):
  def eval(params, training):
    y = v_run_iter_nca(
        train_x, params, wires, training, periodic, num_steps, async_training, key
    )
    return jax.numpy.square(y[..., 0] - train_y[..., 0]).sum()

  return eval(params, 1), {'hard': eval(params, 0)}

val_and_grad = jax.value_and_grad(loss_f, argnums=0, has_aux=True)

# Helper function that can be used for clipping parameters.
upd_f = lambda p: p

@partial(jax.jit, static_argnums=(4, 5, 6))
def train_step(
    train_state, train_x, train_y, wires, periodic, num_steps, async_training
): 
  params, opt_state, key = train_state
  key, k1 = jax.random.split(key, 2)
  (loss, hard), dx = val_and_grad(
      params, wires, train_x, train_y, periodic, num_steps, async_training, k1
  )
  dx, opt_state = opt.update(dx, opt_state, params)
  new_params = optax.apply_updates(params, dx)
  new_params = upd_f(new_params)
  return TrainState(new_params, opt_state, key), loss, hard


In [ ]:
# @title Learning Chekerboard with Asynchronously Training
hyperparams = {'perceive': {}, 'update': {}}
hyperparams['seed'] = 23
hyperparams['lr'] = 0.05
hyperparams['batch_size'] = 1
hyperparams['num_epochs'] = 800
hyperparams['num_steps'] = 50
hyperparams['channels'] = 8
hyperparams['periodic'] = 0
hyperparams['perceive']['n_kernels'] = 16
hyperparams['perceive']['layers'] = [9, 8, 4, 2]
hyperparams['perceive']['connections'] = [
    'first_kernel',
    'unique',
    'unique',
    'unique',
]
init = (
    hyperparams['perceive']['n_kernels']
    * hyperparams['channels']
    * hyperparams['perceive']['layers'][-1]
    + hyperparams['channels']
)
hyperparams['update']['layers'] = (
    [init] + [256] * 14 + [128, 64, 32, 16, 8, hyperparams['channels']]
)
hyperparams['update']['connections'] = ['unique'] * len(
    hyperparams['update']['layers']
)
hyperparams['async_training'] = True



# Create optimizer
opt = optax.chain(
    optax.clip(100.0),  # Clips by value
    optax.adamw(
        learning_rate=hyperparams['lr'], b1=0.9, b2=0.99, weight_decay=1e-2
    ),
)

train_state, wires = init_state(hyperparams, opt, hyperparams['seed'])
key = random.PRNGKey(hyperparams['seed'])

loss_soft = []
loss_hard = []

img = create_checkerboard((TARGET_SIZE_ASYNC, TARGET_SIZE_ASYNC), shape_size=2)

train_y = jnp.zeros(
    shape=(
        hyperparams['batch_size'],
        TARGET_SIZE_ASYNC,
        TARGET_SIZE_ASYNC,
        hyperparams['channels'],
    )
)
train_y = train_y.at[:, :, :, 0].set(img)

for i in range(hyperparams['num_epochs']):
  key, sample_key = random.split(key, 2)
  train_x = random.randint(
      key=sample_key,
      shape=(
          hyperparams['batch_size'],
          TARGET_SIZE_ASYNC,
          TARGET_SIZE_ASYNC,
          hyperparams['channels'],
      ),
      minval=0,
      maxval=2,  # maxval is exclusive
  ).astype(jnp.float32)

  train_state, soft_loss, hard_loss = train_step(
      train_state,
      train_x,
      train_y,
      wires,
      hyperparams['periodic'],
      hyperparams['num_steps'],
      hyperparams['async_training'],
  )
  loss_soft.append(soft_loss)
  loss_hard.append(hard_loss['hard'])

  if i % 100 == 0:
    clear_output(wait=True)
    plot_training_progress(loss_soft, loss_hard, 1)
    print(i, soft_loss, hard_loss['hard'])

clear_output(wait=True)
plot_training_progress(loss_soft, loss_hard, 1, 'checkerboard_async_loss.svg')

In [ ]:
"""Checkerboard pattern with Asynchronous training

Visualization of the recurrent dynamics of the learned circuit.
"""

grid = random.randint(
    key=sample_key,
    shape=(TARGET_SIZE_ASYNC, TARGET_SIZE_ASYNC, hyperparams['channels']),
    minval=0,
    maxval=2,
).astype(jnp.float32)

params, opt_state, key = train_state
wires = wires
training = False
periodic = False

frames = []

nca_state = grid
frames.append(zoom(nca_state[:, :, 0], 16))
for i in range(50):
  key, subkey = jax.random.split(key)
  nca_state = run_async(nca_state, params, wires, training, periodic, subkey)
  frames.append(zoom(nca_state[:, :, 0], 16))

visualize(frames, 'board_async.gif', 5)
Image('board_async.gif')

In [ ]:
"""Checkerboard pattern

Visualization of the recurrent dynamics of the learned circuit
using three different seed to compute asynchrnous masks.
"""

params, opt_state, key = train_state
wires = wires
training = False
periodic = False

frames = []
nca_state_1 = grid.copy()
nca_state_2 = grid.copy()
nca_state_3 = grid.copy()

ZOOM = 32
separator = jnp.zeros((TARGET_SIZE_ASYNC * ZOOM, 5)) + 0.5
frames.append(
    np.hstack([
        zoom(nca_state_1[:, :, 0], ZOOM),
        separator,
        zoom(nca_state_2[:, :, 0], ZOOM),
        separator,
        zoom(nca_state_3[:, :, 0], ZOOM),
    ])
)

for i in range(50):
  key, subkey = jax.random.split(key)
  nca_state_1 = run_async(
      nca_state_1, params, wires, training, periodic, subkey
  )
  key, subkey = jax.random.split(key)
  nca_state_2 = run_async(
      nca_state_2, params, wires, training, periodic, subkey
  )
  key, subkey = jax.random.split(key)
  nca_state_3 = run_async(
      nca_state_3, params, wires, training, periodic, subkey
  )
  frames.append(
      np.hstack([
          zoom(nca_state_1[:, :, 0], ZOOM),
          separator,
          zoom(nca_state_2[:, :, 0], ZOOM),
          separator,
          zoom(nca_state_3[:, :, 0], ZOOM),
      ])
  )

# Display the concatenated video
visualize(frames, 'board_async_multi.gif', 5)
Image('board_async_multi.gif')

In [ ]:
"""Checkerboard pattern with Asynchrnous Training: testing self-healing

We simulate the learned circuit on a grid that is four times
bigger and for four times as many timesteps as during training.

Similar to the previous test by deactivating a 10x10 cell center
to evaluate pattern reconstruction under cell damage.

We now re-activate the cells after 100 steps to evaluate self-healing.
"""

grid = random.randint(
    key=sample_key,
    shape=(TARGET_SIZE_ASYNC * 4, TARGET_SIZE_ASYNC * 4, hyperparams['channels']),
    minval=0,
    maxval=2,
).astype(jnp.float32)

params, opt_state, key = train_state
wires = wires
training = False
periodic = False

print(grid.shape)
frames = []
nca_state = grid
frames.append(zoom(nca_state[:, :, 0], 8))
for i in range(50 * 4):
  key, subkey = jax.random.split(key)
  nca_state = run_async(nca_state, params, wires, training, periodic, subkey)
  if i < 50 * 2:
    nca_state = nca_state.at[
        nca_state.shape[0] // 2 - 10 : nca_state.shape[0] // 2 + 10,
        nca_state.shape[1] // 2 - 10 : nca_state.shape[1] // 2 + 10,
        :,
    ].set(0)

  frames.append(zoom(nca_state[:, :, 0], 8))

visualize(frames, 'healing_board.gif', 5)
Image('healing_board.gif')

In [ ]:
"""Checkerboard pattern: testing robustness

We simulate the learned circuit on a 64x64 grid for 4000 steps.
At each time step, we randomly reset a 10x10 square and measure
the loss function to test how quickly the asynchronously trained
circuit recovers.

We will use the computed err_async to compare it to the synchronously
trained circuit.
"""

grid = random.randint(
    key=sample_key, shape=(64, 64, hyperparams['channels']), minval=0, maxval=2
).astype(jnp.float32)

params, opt_state, key = train_state
wires = wires
training = False
periodic = False
target = create_checkerboard((64, 64), shape_size=2)
frames = []
nca_state = grid
frames.append(zoom(nca_state[:, :, 0], 8))
err_async = []
for i in range(4000):
  key, k1, k2, k3 = random.split(key, 4)
  nca_state = run_async(
      nca_state, params, wires, training, periodic, k1
  )  # async at inference

  index_x = random.randint(k2, shape=(), minval=0, maxval=nca_state.shape[0])
  index_y = random.randint(k3, shape=(), minval=0, maxval=nca_state.shape[0])

  i_x_0 = index_x
  i_x_1 = (
      index_x + 10 if index_x + 10 <= nca_state.shape[0] else nca_state.shape[0]
  )
  i_y_0 = index_y
  i_y_1 = (
      index_y + 10 if index_y + 10 <= nca_state.shape[1] else nca_state.shape[1]
  )
  nca_state = nca_state.at[i_x_0:i_x_1, i_y_0:i_y_1, :].set(0)
  frames.append(zoom(nca_state[:, :, 0], 8))
  err = jax.numpy.abs(nca_state[:, :, 0] - target)
  err_async.append(err.sum())

In [ ]:
"""Testing Robustness

This plot shows the robustness error difference between
synchronous and asynchronous training, as previously calculated.
"""

plt.style.use('seaborn-v0_8-darkgrid')
fig, ax = plt.subplots(figsize=(10, 6), dpi=100)
ax.plot(
    err_async[:4000], color='#2E86C1', linewidth=2, label='Async', alpha=0.9
)

ax.plot(err_sync[:4000], color='#E74C3C', linewidth=2, label='Sync', alpha=0.9)

ax.grid(True, linestyle='--', alpha=0.7)

ax.set_xlabel('Iterations', fontsize=12, fontweight='bold')
ax.set_ylabel('Error', fontsize=12, fontweight='bold')

ax.set_title(
    'Error Comparison - Async vs Sync', fontsize=14, fontweight='bold', pad=20
)

ax.legend(
    frameon=True, fancybox=True, shadow=True, fontsize=10, loc='upper right'
)

for spine in ax.spines.values():
  spine.set_linewidth(1.5)

ax.set_facecolor('#f8f9fa')
fig.set_facecolor('white')

plt.tight_layout()

save_path = 'async_sync.svg'

with open(save_path, 'wb') as f:
  plt.savefig(f, format='svg')

plt.show()